In [ ]:
%load_ext autoreload
%autoreload 2
%matplotlib inline
from IPython.display import display, HTML
from ipywidgets import interact, FloatSlider, IntSlider, Button, Output 
display(HTML("<style>.rendered_html.text_cell_render {max-width:600px; }</style>")) 

# Image classifier

Resourses and useful links:

 - ...

In [ ]:
[1,2,3][:-1]

In [ ]:
#| default_exp nn.image_classifier
#| export
import torch
import torch.nn as nn
import torch.nn.functional as F


class ConvLinClassifier(nn.Module):
    def __init__(self, 
                 imshape, 
                 classes, 
                 channels = [16,32], 
                 kernels  = [3,3], 
                 hidden   = None):
        super().__init__()
        
        
        
        conv_layers = []
        ch          = [imshape[0]] + channels
        for i in range(len(ch)-1): 
            k = 3 if kernels is None else kernels[i]
            conv_layers.extend([
                nn.Conv2d(ch[i], ch[i+1], k), 
                nn.ReLU()])
            
        self.conv = nn.Sequential(
                        *conv_layers, 
                        nn.MaxPool2d(2), 
                        nn.Flatten(1))
        
        with torch.no_grad():
            x = torch.rand(128,*imshape) # b, c, w, h
            x = self.conv(x)
            h0 = x.size(-1)
        
        lin_layers = []
        hs         = [h0] + hidden + [classes]
        for i in range(len(hs)-1):
            lin_layers.extend([ 
                nn.Linear(hs[i], hs[i+1]),
                nn.ReLU()])
        
        
        self.lin  = nn.Sequential(*lin_layers[:-1]) 
        self.out  = nn.Sigmoid() 


    def forward(self, x):
        x = self.conv(x)
        x = self.lin(x)
        x = self.out(x)
        return x
    

In [ ]:
from cgnai.utils import cgnai_home
from torchvision.datasets import MNIST
from torch.utils.data import DataLoader
import torchvision.transforms as transforms
import torch.nn.functional as F
import matplotlib.pyplot as plt

B = 64
mnist  = MNIST(cgnai_home()/"local/mnist", train=True, transform=transforms.ToTensor())
mnist_ = MNIST(cgnai_home()/"local/mnist", train=False, transform=transforms.ToTensor())

def collate(d):
    x,y = zip(*d)
    x = torch.stack(x)
    y = F.one_hot(torch.LongTensor(y),10).float()
    return (x,y)

dl = DataLoader(mnist,  batch_size=B, shuffle=True, collate_fn=collate)
tl = DataLoader(mnist_, batch_size=B, shuffle=True, collate_fn=collate)
x,y = next(iter(dl));

In [ ]:
f = ConvLinClassifier((1,28,28), 10, [16,32], [3,3], [128])


with torch.no_grad():
    x, y  = next(iter(dl))    
    y_hat = f(x)
    
loss = torch.nn.BCELoss()
loss(y_hat, y)
f

In [ ]:
from cgnai.school.training import TrainingLoop, StdBatchLoss

loop = TrainingLoop(
    batch_loss  = StdBatchLoss(f, loss),
    data_loader = dl,
    opt         = torch.optim.Adam
)
loop.run(5);

In [ ]:
plt.plot(loop.L_flatten);

In [ ]:
len(tl)

In [ ]:
import numpy as np
mc = []
with torch.no_grad():
    for x,y in tl:
        y_hat = f(x)
        d_hat = torch.argmax(y_hat, dim=-1)
        d     = torch.argmax(y,     dim=-1)

        # misclassified
        mc.append(torch.sum( d_hat != d)/B)
        
np.mean(mc)